### 1.导入模块

In [20]:
import numpy as np
import time

import seaborn as sns
import pandas as pd
import qulab_toolbox.wavedata as WD
import ctypes
import zerorpc
import math
import datetime

from DC1000 import  DC1000MlutiChannelDevice
from DC2000_16CH_AWG import  DC2000MlutiChannelDevice
from CTP100_Dev_20230713 import ZW_CTP100_Dev
import ZW_AWG1000_dds8

# from qcodes.instrument_drivers.Spectrum.M2p import M2p

import matplotlib.pyplot as plt

### 注意事项 QA设备一定要先远程电脑关系统在断电

#### 修改ip方式

In [21]:
# #### DC1000 DC2000
# ##1.通过机箱背后uart标志的接口接线到电脑上，使用修改IP软件进行修改（软件可联系我们）
# ##2.网口能联通的情况下，指令如下
# DC1000_dev.change_ip("192.168.1.10")  # 改变设备IP,可以不用重启设备,改变后自动生效
# DC2000_dev.change_ip("192.168.1.203")

# #### AWG1000
# ##1.通过机箱背后uart标志的接口接线到电脑上
# awg1000dev_cmd = ZW_AWG1000_dds8.AWG1000()
# awg1000dev_cmd.open_uart("COM3")  # 打开串口3,备后面获取IP
# awg1000dev_cmd.get_dev_ip()       # 
# awg1000dev_cmd.change_dev_ip("uart","192.168.110.100")   #串口修改IP(eth：网口，uart：串口) #***注意修改后，必须重启机箱才能生效 ***#
# ##2.网口能联通的情况下，指令如下
# awg1000dev_cmd = ZW_AWG1000_dds8.AWG1000()
# awg1000dev_cmd.change_dev_ip("eth","192.168.110.100")   #网口修改IP(eth：网口，uart：串口)
# awg1000dev_cmd.get_dev_ip()       # 注意修改后，必须重启机箱才能生效 #

# #### QA1000
# ##1.通过机箱的后面板的HDMI接口和usb接口接显示器鼠标键盘，在网络管理器里直接修改。

# #### CTP100
# ##只能通过网口修改，默认192.168.1.200，***修改后一定要备注记录****
# # 设备当前IP地址变量
# current_ip = '192.168.1.200'
# # 如果想重新设定设备的IP地址变量

# target_ip = '192.168.4.200'
# # 如果想重新设定设备MAC地址变量

# target_mac = '00.10.35.240.00.18'
# # target_mac = '00.10.35.240.00.19'
# # 设定设备到新的IP地址，参数当前IP在前，目标IP地址在后

# dev_ctp.set_dev_ip(current_ip,target_ip)
# # 设定设备到新的MAC地址，参数当前IP在前，目标MAC地址在后

# dev_ctp.set_dev_mac(current_ip,target_mac)
# # 根据设备地址，获取设备信息，返回设备名称、设备IP以及设备MAC地址

# dev_ctp.read_system_ip(current_ip)

#### 环境搭建

In [ ]:
##1.准备一台铷钟，将铷钟的输出10MHz接到AWG1000/QA1000/CTP100的REFIN,注意先接好线且铷钟面板时钟锁定灯亮了，再开机AWG/QA/CTP等设备
##2.将CTP的T1-T9分别接到DC2000/AWG1000/QA1000的trig_in接口（注意DC2000是IN1接口）

### 2.连接仪器 和初始化

In [22]:
#DC2000 CONNECT
DC2000_dev = DC2000MlutiChannelDevice("DC2000", "192.168.1.201")

#DC1000 CONNECT
DC1000_dev = DC1000MlutiChannelDevice("DC1000", "192.168.1.10")
# 网络连接CTP100设备
# 确保控制计算机包含目标CTP100设备的默认IP网络端内IP地址
dev_ctp = ZW_CTP100_Dev('192.168.1.200')

#AWG1000 CONNECT
awg1000dev_data = ZW_AWG1000_dds8.AWG1000()
awg1000dev_cmd = ZW_AWG1000_dds8.AWG1000()

awg1000dev_cmd.connect("192.168.1.104", 9001)
awg1000dev_data.connect("192.168.1.104", 9002)

#QA1000 CONNECT
server_ip = '192.168.1.180:8501'

qa1000dev = zerorpc.Client(timeout=60, heartbeat=None)

try:
    qa1000dev.connect('tcp://' + server_ip)
    if qa1000dev.my_status() in ['ok']:
        print('rpc connect ok')
except:
    print('rpc connect error')

Connection error: timed out


TimeoutError: timed out

In [24]:
#AWG1000 SET WORK MODE 外参考模式
awg1000dev_cmd.set_refclk("ext_ref", 10)  # 设置内外参考
time.sleep(5)

#DC2000 SET INIT
DC2000_dev.ch1.mode('AWG')              # 设置模式
DC2000_dev.ch1.output_state('ON')       # 打开CH2
DC2000_dev.ch1.enable_trig(True)        # CH2使能触发
DC2000_dev.enable_global_trig(True)     # 设备使能全局触发
DC2000_dev.all_ch_status()              # 获取所有通道状态
#DC1000 SET INIT
DC1000_dev.allch_status() # 查看所有通道状态
#QA1000 SET INIT
#DA Nyquist域，1: 第二奈奎斯特域，0：第一奈奎斯特域
Nyquist = 0
qa1000dev.rfdac_SetNyquistZone(Nyquist)
qa1000dev.rfadc_SetNyquistZone(Nyquist)

NameError: name 'awg1000dev_cmd' is not defined

### DC2000_NORMAL模式

In [7]:
DC2000_dev.ch1.mode('NORMAL')           # 设置模式
DC2000_dev.ch2.mode('NORMAL')           # 设置模式

In [8]:
DC2000_dev.ch1.slope(10000)             # 设置CH1 斜率，单位：mV/s
DC2000_dev.ch2.slope(10000)             # 设置CH2 斜率，单位：mV/s

In [9]:
DC2000_dev.ch1.pwm(False)                # 使能CH1 PWM 功能
DC2000_dev.ch2.pwm(False)                # 使能CH2 PWM 功能

In [10]:
DC2000_dev.ch1.output_state('ON')       # 打开CH1
DC2000_dev.ch2.output_state('ON')       # 打开CH2

In [ ]:
#dc2000 通道1、2 遍历扫一个二维图，踢掉采集部分 示例代码
DC2000_dev.ch2.voltage(0)              # 设置CH1 电压，单位：V

x = []
y = []
z = []
z1 = []

time.sleep(1)

for i in np.arange(-1.0, 1.1, 0.1):
    y.append(round(i,6))

for nx in np.arange(-1.0, 1.1, 0.1):
    x.append(round(nx,6))
    DC2000_dev.ch2.voltage(nx)
    for i in np.arange(-1.0, 1.1, 0.1):
        DC2000_dev.ch1.voltage(i)
        # a = m2p.single_software_trigger_acquisition(2000, int(2e3), int(2e3))
        time.sleep(0.01) # 延时10ms
        ch2 = 10
        ztmp = np.mean(ch2)
        z1.append(ztmp)
    z.append(z1)
    z1 = []

df = pd.DataFrame(data=z, index=y,columns=x)
# now_time = datetime.datetime.now().strftime('%H_%M_%S')

sns.heatmap(df,cmap="viridis_r")
plt.title("DC2000")
plt.xlabel('CH1')
plt.ylabel('CH2')
# plt.savefig(f'./data_qcs/Zbias_cavityfreq_sweep/{QUbit}_{now_time}.png')
plt.show()



In [5]:
DC2000_dev.ch1.output_state('OFF')      # 关闭CH1
DC2000_dev.ch2.output_state('OFF')      # 关闭CH1

### DC2000_AWG模式

In [ ]:
DC2000_dev.ch1.mode('AWG')              # 设置模式
DC2000_dev.all_ch_status()              # 获取所有通道状态

In [ ]:
### ctp触发设置

# 重新配置参数生效之前，先关闭所有通道trig输出
dev_ctp.trigger_close()
trigger_time = 0
trigger_parameter = []
for i in range(11):
    trigger_parameter.append({'trigger_source':0,'trigger_continue':0,'trigger_block_en':1,'trigger_ref':0,
                              'trigger_times':2,'trigger_us':5,'trigger_delay':i*trigger_time})
        #trigger_parameter = {'trigger_source':0,'trigger_continue':0,'trigger_times':0,'trigger_us':0,'trigger_delay':0}
        #trigger_source 0 为内部触发，1为外部触发
        #trigger_us 触发周期单位us
        #trigger_num 触发次数，当trigger_continue为1时无效
        #trigger_continue 1 为连续触发模式，此时触发次数无效；0 为按触发次数触发。
        #trigger_delay 触发相对延时单位us

# trigger_parameter[2]['trigger_delay'] = 1000
# trigger_parameter[4]['trigger_delay'] = 1000

# 按照先前设定的各个通道trig工作参数，逐个通道进行参数写入
for trigger_ch in range(11):
    dev_ctp.trigger_ctrl(trigger_ch+1,trigger_parameter[trigger_ch])

##DC2000设置
#以下两种方式二选一

# 方式1
point_num = 16384                   # 最大32K
sample_fs = 0.5e+6                  # 采样频率
signal_fs = 0.5e+6/(128*1)          # 信号频率
n = point_num 
n = np.arange(n)
y = np.sin(2*np.pi*signal_fs*n/sample_fs) * 0.9             # 满幅值20Vpp, 0.9为幅度系数
wave = np.append(y,0)
DC2000_dev.ch1.wave(wave) 
wave_len = int(len(wave))                                         # 下载波形数据
DC2000_dev.ch1.awg_wave_param(np.asarray((1, 2, wave_len)))     # 设置波形参数 param1 :0 连续播放 1：trig 播放 param2:播放次数

# wave = []
# point_num = 16384                   # 最大32K
# sample_fs = 0.5e+6                  # 采样频率
# signal_fs = 0.5e+6/(128*1)          # 信号频率
# n = point_num 
# n = np.arange(n)
# # y = np.sin(2*np.pi*signal_fs*n/sample_fs) * 0.9             # 满幅值20Vpp, 0.9为幅度系数
# y=np.ones(1000)*((1)/10)
# wave.append(y)
# y=np.ones(1000)*((2)/10)
# wave.append(y)
# y=np.ones(1000)*((3)/10)
# wave.append(y)
# y=np.ones(1000)*((4)/10)
# wave.append(y)
# wave = list(np.concatenate(wave))
# wave = np.array(wave)      # 波形数据生成
# DC2000_dev.ch1.wave(wave)  # load波形数据
# wave_len = int(len(wave))                                       
# DC2000_dev.ch1.awg_wave_param(np.asarray((1, 1, wave_len)))     # 设置波形参数 param1 :0 连续播放 1：trig 播放 param2:播放次数 param:播放长度
plt.plot(wave)

# 方式2
# mode = 0x02     # 01 ：sine ；02：square； 03:triangle 04:sawtooth 05 : pulse
# duty = 0.5       # 脉冲模式占空比，范围：0< plus < 1（仅用于脉冲输出模式）
# freq = 1000     # 输出频率 单位：Hz
# amp = 2500      # 输出幅度 单位:mV
# phase = 0       # 相位 单位：°（仅用于sine波输出模式）
# offset = 0      # 偏置 单位:mV
# device.ch1.predefined_wave_param(np.asarray((mode, duty, freq, amp, phase, offset)))     # 预定义波形数据 

In [8]:
DC2000_dev.ch1.output_state('ON')       # 打开CH1

In [9]:
DC2000_dev.ch1.enable_trig(True)        # CH1使能触发

In [11]:
DC2000_dev.soft_trig(np.asarray((1,1))) # 软触发 （数量，间隔s）

In [76]:
DC2000_dev.ch1.output_state('OFF')      # 关闭CH1

In [ ]:
# 开启trig通道输出使能
dev_ctp.trigger_open()

### DC1000控制

In [ ]:
DC1000_dev.allch_status() # 查看所有通道状态

In [26]:
DC1000_dev.ch2.slope(5000) # 设置电压上升下降的斜率,有默认值1V/S,可不设置
DC1000_dev.ch2.output_state('ON') # 打开通道2


In [28]:
DC1000_dev.ch2.voltage(-10) # 通道2设置电压10V

In [29]:
DC1000_dev.ch2.output_state('OFF') # 关闭通道2

### AWG1000 AWG模式输出

In [ ]:
#awg设备输出两段sin300M 800ns的波形，通过示波器查看
### ctp触发设置

# 重新配置参数生效之前，先关闭所有通道trig输出
dev_ctp.trigger_close()
trigger_time = 0
trigger_parameter = []
for i in range(11):
    trigger_parameter.append({'trigger_source':0,'trigger_continue':0,'trigger_block_en':1,'trigger_ref':0,
                              'trigger_times':2,'trigger_us':5,'trigger_delay':i*trigger_time})
        #trigger_parameter = {'trigger_source':0,'trigger_continue':0,'trigger_times':0,'trigger_us':0,'trigger_delay':0}
        #trigger_source 0 为内部触发，1为外部触发
        #trigger_us 触发周期单位us
        #trigger_num 触发次数，当trigger_continue为1时无效
        #trigger_continue 1 为连续触发模式，此时触发次数无效；0 为按触发次数触发。
        #trigger_delay 触发相对延时单位us

# trigger_parameter[2]['trigger_delay'] = 1000
# trigger_parameter[4]['trigger_delay'] = 1000

# 按照先前设定的各个通道trig工作参数，逐个通道进行参数写入
for trigger_ch in range(11):
    dev_ctp.trigger_ctrl(trigger_ch+1,trigger_parameter[trigger_ch])

### awg设备设置
awg_ch = 2
awg_mode = 0 
awg1000dev_cmd.set_data_source(awg_ch, 50, awg_mode)  # 设置模式为自主波形模式 参数1：50不用关心

awg1000dev_cmd.set_output_mode(awg_ch, "AC")  # 通道1设置AC模式输出
awg1000dev_cmd.set_trigger_param(awg_ch, 0, 0, 0)  # 通道1设置触发参数
awg1000dev_cmd.close_channel([awg_ch])#关闭通道

wave_data = awg1000dev_cmd.sin_wave(1,300e+6,0,800e-9) #生成波形数据

status = awg1000dev_data.send_waveform_data(wave_data,awg_ch) #load波形
print(f'通道{awg_ch}数据下载成功') if status else print(f'通道{awg_ch}数据下载失败')
time.sleep(0.1) 
awg1000dev_cmd.open_channel([awg_ch])  # 打开通道    

# 开启trig通道输出使能
dev_ctp.trigger_open()

### AWG DDS模式

In [42]:
#控制AWG ch2为DDS模式控制其输出 单音3段序列第一段为40M250ns的波形，第二段为2us的0（等效于间隔），第三段为2us的100M sin波
##CTP100设备trig参数设置
trigger_time = 0
trigger_parameter = []
for i in range(11):
    trigger_parameter.append({'trigger_source':0,'trigger_continue':0,'trigger_block_en':1,'trigger_ref':0,
                              'trigger_times':1,'trigger_us':5,'trigger_delay':i*trigger_time})

# 重新配置参数生效之前，先关闭所有通道trig输出
dev_ctp.trigger_close()

# trigger_parameter[2]['trigger_delay'] = 1000
# trigger_parameter[4]['trigger_delay'] = 1000

# 按照先前设定的各个通道trig工作参数，逐个通道进行参数写入
for trigger_ch in range(11):
    dev_ctp.trigger_ctrl(trigger_ch+1,trigger_parameter[trigger_ch])


#SET AWG1000
outch = 2
dds_mode = 3
awg1000dev_cmd.set_data_source(outch, 50, dds_mode)  # 设置模式为自主波形模式 参数1：50不用关心
awg1000dev_cmd.close_channel([outch])  # 关闭通道

awg1000dev_cmd.set_output_mode(outch, "AC")  # 通道1设置AC模式输出
awg1000dev_cmd.set_trigger_param(outch, 0, 0, 0)  # 通道设置触发参数
#单通道输出单波形，n个序列
trigdelay = [100] #dds输出延迟值
paramx1 = [] #x8表示可以出 8音信号 每一音可以出1024个波形序列
paramx2 = []
paramx3 = []
paramx4 = []
paramx5 = []
paramx6 = []
paramx7 = []
paramx8 = []
# paramx1.append([freq, length, amp, phase_offest, win_type, win_len])#length设置为0时表示输出连续波
pw = 250e-9
fs = 2.4e9
amp = 0.5
paramx1.append([40*1e6,round(pw*fs),round(65535*amp),0,0,round(pw*fs)]) #第一段序列
paramx1.append([30*1e6,5000,round(65535*0),90,0,200])     #第二段序列
paramx1.append([100*1e6,5000,round(65535*amp),90,0,200])  #第三段序列
paramx8 = []
paramx8.append(paramx1)
awg1000dev_cmd.DefWaveGen(outch , trigdelay, paramx8)#load序列参数


time.sleep(0.1) 
#开始给trig
dev_ctp.trigger_open()

time.sleep(0.1) 

### QA1000 DA输出函数

In [32]:
def qa1000_set_freq(chennel_num, fout, datalen_s, amp=1, delay = 0,phi = 0 ):
    #dac setting
    sampleRate=4e9
    phase_s = phi    #初相
    fout = fout*1e6   #频率
    replay_len = round(datalen_s*sampleRate)  #波形长度，单位点数，da采样率4G，每点250ps
    amp = 1   #幅度

    dac_data=[]
    dac_data.append(phase_s)
    dac_data.append(fout)
    dac_data.append(replay_len)
    dac_data.append(amp)

    trigger_delay = round(delay*sampleRate)  #波形长度，单位点数，da采样率4G，每点250ps
    replay_times = 1000000   #da输出次数，为负时一直输出
    # print(trigger_delay,replay_len,fout)
    chennel_num = chennel_num
    qa1000dev.dac_updata(chennel_num,trigger_delay,replay_times,dac_data)

### QA1000 原始数据采集

In [ ]:
### 读取设备输出通道1和采集通道1 接电感板和另外一个模块测试S21的原始波形结果
#INIT TRIGER ctp100设备trig初始化
trigger_time = 0
trigger_parameter = []
for i in range(11):
    trigger_parameter.append({'trigger_source':0,'trigger_continue':0,'trigger_block_en':1,'trigger_ref':0,
                              'trigger_times':1,'trigger_us':1000,'trigger_delay':i*trigger_time})

# 重新配置参数生效之前，先关闭所有通道trig输出
dev_ctp.trigger_close()

# trigger_parameter[2]['trigger_delay'] = 1000
# trigger_parameter[4]['trigger_delay'] = 1000 #shiji 800us
# trigger_parameter[6]['trigger_delay'] = 100

# 按照先前设定的各个通道trig工作参数，逐个通道进行参数写入
for trigger_ch in range(11):
    dev_ctp.trigger_ctrl(trigger_ch+1,trigger_parameter[trigger_ch])

#QA1000 setting
#ADC setting
# 设定触发来源
trigger_source = 1  # 触发来源，1为外部触发，0为内部触发
# 如果是外部触发，如下参数不起效
trigger_us = 500  # 内部触发周期，us
trigger_num = 10  # 内部触发次数
trigger_continue = 1  # 内部触发连续触发模式，1开启，0关闭

ad_trigger_delay = 1100  # AD采集延迟，样点数量，用户可以根据采样率5Gsps和时间计算取整后得到
times = 1  # AD采集次数
save_len = 2**16  # AD采集长度，单位点数，ad采样率5G，每点200ps

#关闭触发响应

qa1000dev.trigger_close()
# 写入ad工作参数
for adc_channel_num in range(4):
    qa1000dev.rd_adc_data_ctrl(adc_channel_num, ad_trigger_delay, times, save_len)

#dac setting
qa1000_set_freq(chennel_num = 0, fout = 249, datalen_s=2e-6, amp=1, delay = 0,phi = 0 )

#打开触发响应
qa1000dev.trigger_ctrl(trigger_source,trigger_us,trigger_num,trigger_continue)

# 开启trig通道输出使能
dev_ctp.trigger_open()

time.sleep(0.1) 

adc_data = []
#读取ad采集到的波形数据
for adc_channel_num in range(4):
    adc_data_buf, save_len_i = qa1000dev.rd_adc_data(adc_channel_num, times, save_len)
    adc_data.append(np.array(adc_data_buf))

# 绘出所有通道时域波形
plt.figure()
# for adc_channel_num in range(4):
#     # plt.plot(adc_data[adc_channel_num][900:950])
#     plt.plot(adc_data[adc_channel_num][:])

# 绘制某一个通道ADC采集数据FFT波形
adc_channel_num = 0

plt.figure()
plt.plot(adc_data[adc_channel_num][:800])

In [ ]:
# sampleRate=5e9

# data_len = 600
# data_len_t = data_len/sampleRate

# fout = 250e6

# sin1=WD.Sin(fout*2*np.pi,0,data_len_t,sampleRate)
# cos1=WD.Cos(fout*2*np.pi,0,data_len_t,sampleRate)

# datai = 0
# dataq = 0
# for i in range(data_len):
#     datai += adc_data[adc_channel_num][i] * cos1.data[i] 
#     dataq += adc_data[adc_channel_num][i] * sin1.data[i]

# print(datai, dataq, abs(datai + dataq * 1j))

### QA1000（常温）扫腔频实验

In [36]:
### 读取设备输出通道1和采集通道1 接电感板和另外一个模块测试S21的吸收频率范围
def sweep_freq(param):
    for i in range(4):
        qa1000dev.dac_close(i)
        qa1000_set_freq(i, 0, 2e-6, 0, delay=0e-9)
        
    temp_data = [] 
    y_mod = []

    trigger_source = 1    #触发来源，1为外部触发，0为内部触发
    trigger_us = param['trig_period']      #内部触发周期，us
    trigger_continue = 0  #内部触发连续触发模式，1开启，0关闭

    #开启触发信号使能，此时等待外部触发信号触发DA输出与AD采集
    trigger_parameter = {
        'trigger_source':0,
        'trigger_continue':trigger_continue,
        'trigger_block_en':1,
        'trigger_ref':0,
        'trigger_times':param['shots'],
        'trigger_us':trigger_us,
        'trigger_delay':0
    }
    dev_ctp.trigger_ctrl(param['trig_ch'], trigger_parameter)

    x = np.around(np.linspace(param['sweep_freq']['start'], param['sweep_freq']['end'], 
                              round(abs(param['sweep_freq']['end'] - param['sweep_freq']['start'])/param['sweep_freq']['step']) + 1), 2)
    for freq in x:
        qa1000dev.trigger_close()
        dev_ctp.trigger_close()

        qa1000_set_freq(param['read_out_ch'], freq, param['read_out_pw'], amp=param['read_out_amp'],delay=param['read_out_delay'])




        #解模参数配置
        adsampleRate = 5e9
        ad_trigger_delay = (238e-9 + param['read_in_delay'])*adsampleRate  #AD采集延迟，样点数量，用户可以根据采样率5Gsps和时间计算取整后得到

        mul_f = []              #解模参数列表
        mul_start_phase = 0     #解模参数：初相
        mul_f_freq = (freq) * 1e6    #解模参数：频率，一般每个qubit对应不同频率
        mul_f_len = round(param['read_in_pw'] * adsampleRate)     #解模参数：单次解模shot数据样点数量，单位点数，ad采样率5G，每点200ps

        mul_f.append(mul_start_phase)
        mul_f.append(mul_f_freq)       #解模频率，实际应当按照各个qubit设定
        mul_f.append(mul_f_len)        #单次解模shot数据样点数量



        qa1000dev.rd_adc_mul_data_ctrl(param['read_in_ch'],ad_trigger_delay,param['shots'],mul_f)
        qa1000dev.trigger_ctrl(trigger_source,trigger_us,param['shots'],trigger_continue)

        dev_ctp.trigger_open()

        #稍作延迟
        time.sleep(0.001)

        # print(22)
        while 1:
            mul_data_bufe,read_data_len = qa1000dev.rd_adc_mul_data(param['read_in_ch'],0,param['shots'])
            if read_data_len != -1:
                break
            
        for i in range(read_data_len):
            mul_data_bufe[i] = complex(mul_data_bufe[i])
        # print(mul_data_bufe)
        temp_data.append(np.mean(mul_data_bufe))
        y_mod = np.abs(np.asarray(temp_data)/ mul_f_len)
        # print(y_mod)
    return x, y_mod



In [ ]:
param = {
    "sweep_freq": {"start":50,"end":400,"step":2}, ##MHz 读取输出的扫描范围
    "read_out_ch": 0, #读取输出的通道
    "read_in_ch": 0,  #读取输入的通道
    "read_in_pw":600e-9, #读取输入的脉宽 ns
    "read_out_pw":600e-9,#读取输出的脉宽 ns
    "read_out_amp":1, #读取输输出的幅度 （归一化）
    "read_out_delay":0e-9,#读取输出的延时
    "read_in_delay":0e-9,#读取输入的延时
    "shots":50, #读取测试次数
    "trig_period":50, ##utrig的周期
    "trig_ch":9,#读取输入的通道trig的通道号
}
QUbit = 'balun_board'
x, y = sweep_freq(param)
#绘图和数据保存
now_time = datetime.datetime.now().strftime('%H_%M_%S')
plt.plot(x,y)
plt.title("cavity_freq")
plt.xlabel('MHz')
plt.ylabel('mod')#
plt.savefig(f'./data/cavity_freq/{QUbit}_{now_time}.png')
plt.show()
d = {'x_cavity_freq':x, 'y_mod':y}
df = pd.DataFrame(data=d)
df.to_csv(f'./data/cavity_freq/{QUbit}_{now_time}.csv')
maxmodindex = np.argmin(y)
print(x[maxmodindex])

### AWG-QA扫腔（常温）耗时测试


In [ ]:
### 通过控制AWG和QA扫腔频实验，验证指令下载的耗时 输出结果图1为扫腔结果，图2为每个频率下发和波形输出完成的耗时波动
##设置AWG设备和QA设备不同的trig参数
short = 10
trigger_source = 1    #触发来源，1为外部触发，0为内部触发
dev_ctp.trigger_close()
trigger_parameter_awg = {
    'trigger_source':0,
    'trigger_continue':0,
    'trigger_block_en':1,
    'trigger_ref':0,
    'trigger_times':1,
    'trigger_us':5000,#5ms
    'trigger_delay':0
}
dev_ctp.trigger_ctrl(7, trigger_parameter_awg)

trigger_parameter_qa1000 = {
    'trigger_source':0,
    'trigger_continue':0,
    'trigger_block_en':1,
    'trigger_ref':0,
    'trigger_times':short,
    'trigger_us':1000,#5ms
    'trigger_delay':0
}
dev_ctp.trigger_ctrl(9, trigger_parameter_qa1000)
#SET AWG1000
awg_mode = 0 
dds_mode = 3
awg1000dev_cmd.set_data_source(1, 50, dds_mode)  # 设置模式为自主波形模式
awg1000dev_cmd.close_channel([1,2,3,4,5,6,7,8])  # 关闭通道

for i in range(1,9,1):
    awg1000dev_cmd.set_output_mode(i, "AC")  # 通道1设置AC模式输出
for i in range(1,9,1):
    awg1000dev_cmd.set_trigger_param(i, 0, 0, 0)  # 通道1-8设置触发参数

x = np.around(np.linspace(220, 270, round(abs(270 - 220)/1) + 1), 2)

y_mod = []
temp_data = [] 
time_data = [] 

for freq in x :
    now_time = datetime.datetime.now().strftime('%H_%M_%S.%f')
    ms_datas = datetime.datetime.now().strftime('%f')
    print(f'time-start_{freq}MHz:{now_time}')
    #单通道输出单波形，n个序列
    trigdelay = [1]
    paramx1 = []
    paramx2 = []
    paramx3 = []
    paramx4 = []
    paramx5 = []
    paramx6 = []
    paramx7 = []
    paramx8 = []
    # paramx1.append([freq, length, amp, phase_offest, win_type, win_len])
    pw = 500e-6
    fs = 2.4e9
    amp = 1
    paramx1.append([freq*1e6,round(pw*fs),round(65535*amp),0,0,round(pw*fs)])
    paramx1.append([freq*1e6,round(pw*fs),round(65535*0  ),0,0,round(pw*fs)])    
    paramx1.append([freq*1e6,round(pw*fs),round(65535*amp),0,0,round(pw*fs)])
    paramx1.append([freq*1e6,round(pw*fs),round(65535*0  ),0,0,round(pw*fs)]) 
    paramx1.append([freq*1e6,round(pw*fs),round(65535*amp),0,0,round(pw*fs)])
    paramx1.append([freq*1e6,round(pw*fs),round(65535*0  ),0,0,round(pw*fs)])     
    paramx1.append([freq*1e6,round(pw*fs),round(65535*amp),0,0,round(pw*fs)])
    paramx1.append([freq*1e6,round(pw*fs),round(65535*0  ),0,0,round(pw*fs)])    
    paramx1.append([freq*1e6,round(pw*fs),round(65535*amp),0,0,round(pw*fs)])
    paramx1.append([freq*1e6,round(pw*fs),round(65535*0  ),0,0,round(pw*fs)]) 
    paramx1.append([freq*1e6,round(pw*fs),round(65535*amp),0,0,round(pw*fs)])
    paramx1.append([freq*1e6,round(pw*fs),round(65535*0  ),0,0,round(pw*fs)])  
    paramx1.append([freq*1e6,round(pw*fs),round(65535*amp),0,0,round(pw*fs)])
    paramx1.append([freq*1e6,round(pw*fs),round(65535*0  ),0,0,round(pw*fs)])    
    paramx1.append([freq*1e6,round(pw*fs),round(65535*amp),0,0,round(pw*fs)])
    paramx1.append([freq*1e6,round(pw*fs),round(65535*0  ),0,0,round(pw*fs)]) 
    paramx1.append([freq*1e6,round(pw*fs),round(65535*amp),0,0,round(pw*fs)])
    paramx1.append([freq*1e6,round(pw*fs),round(65535*0  ),0,0,round(pw*fs)])  
    paramx1.append([freq*1e6,round(pw*fs),round(65535*amp),0,0,round(pw*fs)])
    paramx1.append([freq*1e6,round(pw*fs),round(65535*0  ),0,0,round(pw*fs)])  
    paramx8 = []
    paramx8.append(paramx1)

    out200m_ch = 1
    awg1000dev_cmd.DefWaveGen(out200m_ch , trigdelay, paramx8)
    awg1000dev_cmd.DefWaveGen(2 , trigdelay, paramx8)#通道2也输出，方便示波器看输出
    now_time_da = datetime.datetime.now().strftime('%H_%M_%S.%f')
    print(f'time-param_da_{freq}MHz:{now_time_da}')   

    # #解模参数配置
    adsampleRate = 5e9
    #外触发无意义
    trigger_us = 500  # 内部触发周期，us
    trigger_num = 10  # 内部触发次数
    trigger_continue = 1  # 内部触发连续触发模式，1开启，0关闭

    ad_trigger_delay = (300e-9 + 0)*adsampleRate  #AD采集延迟，样点数量，用户可以根据采样率5Gsps和时间计算取整后得到

    mul_f = []              #解模参数列表
    mul_start_phase = 0     #解模参数：初相
    mul_f_freq = (freq) * 1e6    #解模参数：频率，一般每个qubit对应不同频率
    mul_f_len = round(1000e-9 * adsampleRate)     #解模参数：单次解模shot数据样点数量，单位点数，ad采样率5G，每点200ps

    mul_f.append(mul_start_phase)
    mul_f.append(mul_f_freq)       #解模频率，实际应当按照各个qubit设定
    mul_f.append(mul_f_len)        #单次解模shot数据样点数量

    qa1000dev.rd_adc_mul_data_ctrl(0,ad_trigger_delay,short,mul_f)
    qa1000dev.trigger_ctrl(trigger_source,trigger_us,short,trigger_continue)

    now_time_m = datetime.datetime.now().strftime('%H_%M_%S.%f')
    print(f'time-param_ad{freq}MHz:{now_time_m}')    
    dev_ctp.trigger_open()

    #稍作延迟
    time.sleep(0.001)

    while 1:
        mul_data_bufe,read_data_len = qa1000dev.rd_adc_mul_data(0,0,short)
        if read_data_len != -1:
            break
        
    for i in range(read_data_len):
        mul_data_bufe[i] = complex(mul_data_bufe[i])
    
    temp_data.append(np.mean(mul_data_bufe))
    y_mod = np.abs(np.asarray(temp_data)/ mul_f_len)

    now_time_end = datetime.datetime.now().strftime('%H_%M_%S.%f')
    ms_datae = datetime.datetime.now().strftime('%f')
    print(f'time-end_{freq}MHz:{now_time_end}')
    if (int(ms_datae)-int(ms_datas)>0):
        time_data.append(int(ms_datae)-int(ms_datas))
    else:
        time_data.append((int(ms_datae)+1000000)-int(ms_datas))

plt.figure(0)
plt.plot(y_mod)
plt.figure(1)
plt.plot(time_data)
